In [4]:
import pandas as pd
import numpy as np

from pandas.plotting import scatter_matrix

import matplotlib.pyplot as plt
import plotly.plotly as py

import seaborn as sns 

from sklearn.linear_model import LogisticRegression

from scipy.sparse import csr_matrix

import patsy as pt
from patsy import dmatrices
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

from pygam import PoissonGAM
from pygam.utils import generate_X_grid

from sklearn.metrics import accuracy_score

import graphlab

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



In [5]:
pd.set_option("display.max_rows",500)

In [6]:
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

In [7]:
graphlab.canvas.set_target('ipynb')

In [8]:
prior = pd.read_csv('/Users/mengruiyin/Desktop/project/prior.csv')

In [9]:
prior_data_da = pd.read_csv('/Users/mengruiyin/Desktop/project/prior_data_da.csv')

In [10]:
train = pd.read_csv('/Users/mengruiyin/Desktop/project/train.csv')
train_data = pd.read_csv('/Users/mengruiyin/Desktop/project/train_data.csv')
train_data_da = pd.read_csv('/Users/mengruiyin/Desktop/project/tran_data_da.csv')

In [9]:
train_data_da.head(100)

Unnamed: 0  order_id  user_id eval_set  order_number  order_dow  \
0            0   1187899        1    train            11          4   
1            1   1187899        1    train            11          4   
2            2   1187899        1    train            11          4   
3            3   1187899        1    train            11          4   
4            4   1187899        1    train            11          4   
5            5   1187899        1    train            11          4   
6            6   1187899        1    train            11          4   
7            7   1187899        1    train            11          4   
8            8   1187899        1    train            11          4   
9            9   1187899        1    train            11          4   
10          10   1187899        1    train            11          4   
11          11   1492625        2    train            15          1   
12          12   1492625        2    train            15          1   
13          13   1492625        2    train            15          1   
14          14   1492625        2    train            15          1   
15          15   1492625        2    train            15          1   
16          16   1492625        2    train            15          1   
17          17   1492625        2    train            15          1   
18          18   1492625        2    train            15          1   
19          19   1492625        2    train            15          1   
20          20   1492625        2    train            15          1   
21          21   1492625        2    train            15          1   
22          22   1492625        2    train            15          1   
23          23   1492625        2    train            15          1   
24          24   1492625        2    train            15          1   
25          25   1492625        2    train            15          1   
26          26   1492625        2    train            15          1   
27          27   1492625        2    train            15          1   
28          28   1492625        2    train            15          1   
29          29   1492625        2    train            15          1   
30          30   1492625        2    train            15          1   
31          31   1492625        2    train            15          1   
32          32   1492625        2    train            15          1   
33          33   1492625        2    train            15          1   
34          34   1492625        2    train            15          1   
35          35   1492625        2    train            15          1   
36          36   1492625        2    train            15          1   
37          37   1492625        2    train            15          1   
38          38   1492625        2    train            15          1   
39          39   1492625        2    train            15          1   
40          40   1492625        2    train            15          1   
41          41   1492625        2    train            15          1   
42          42   2196797        5    train             5          0   
43          43   2196797        5    train             5          0   
44          44   2196797        5    train             5          0   
45          45   2196797        5    train             5          0   
46          46   2196797        5    train             5          0   
47          47   2196797        5    train             5          0   
48          48   2196797        5    train             5          0   
49          49   2196797        5    train             5          0   
50          50   2196797        5    train             5          0   
51          51    525192        7    train            21          2   
52          52    525192        7    train            21          2   
53          53    525192        7    train            21          2   
54          54    525192        7    train            21          2   
55          55    525192    

In [10]:
# transform_data = train_data_da[['user_id','order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order',
#                                 'add_to_cart_order', 'reordered', 'department', 'aisle']]
# conditions = [(transform_data['days_since_prior_order'] >=0) & (transform_data['days_since_prior_order'] <= 7),
#              (transform_data['days_since_prior_order'] >=8) & (transform_data['days_since_prior_order'] <= 14),
#              (transform_data['days_since_prior_order'] >=15) & (transform_data['days_since_prior_order'] <= 21),
#               (transform_data['days_since_prior_order'] >=22) & (transform_data['days_since_prior_order'] <= 30)]
# choices = [0, 1, 2, 3] 
# transform_data.loc[:,('response')] = np.select(conditions, choices, default=np.nan)
# product_number = pd.DataFrame(train_data_da.groupby(['user_id'])['product_name'].count()).reset_index()

In [8]:
cluster_day = pd.read_csv('/Users/mengruiyin/Desktop/project/cluster_day.csv')[['user_id', 'day']]
cluster_time = pd.read_csv('/Users/mengruiyin/Desktop/project/cluster_time.csv')[['user_id','time']]
department = pd.read_csv('/Users/mengruiyin/Desktop/project/gensim/department_list.csv')[['user_id', 'topic']]
aisle = pd.read_csv('/Users/mengruiyin/Desktop/project/gensim/aisle_list.csv')[['user_id', 'topic']]

## Method 1

In [9]:
from statsmodels.tools import categorical
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV
from pprint import pprint

This non-commercial license of GraphLab Create for academic use is assigned to my102@duke.edu and will expire on May 31, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1520888251.log
/Users/mengruiyin/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.



In [10]:
def build_feature(df):
    X = df[['order_number', 'product_number', 'order_dow', 'order_hour_of_day']]
    #day = categorical(df['day'].as_matrix(), drop = True)
    #time = categorical(df['time'].as_matrix(), drop = True)
    department = categorical(df['department'].as_matrix(), drop = True)
    aisle = categorical(df['aisle'].as_matrix(), drop = True)
    return np.concatenate((X, department, aisle), axis = 1)
   
    

In [11]:
product_number = pd.DataFrame(prior_data_da.groupby(['user_id'])['product_name'].count()).reset_index()

user_id  product_name
0             1            59
1             2           195
2             3            88
3             4            18
4             5            37
5             6            14
6             7           206
7             8            49
8             9            76
9            10           143
10           11            94
11           12            74
12           13            81
13           14           210
14           15            72
15           16            70
16           17           294
17           18            39
18           19           204
19           20            22
20           21           205
21           22            69
22           23            40
23           24            38
24           25            21
25           26            56
26           27           768
27           28           183
28           29           225
29           30            11
30           31           299
31           32           114
32           33            53
33           34            32
34           35           184
35           36           153
36           37           188
37           38           195
38           39           141
39           40           104
40           41            65
41           42           133
42           43           147
43           44            37
44           45            18
45           46           157
46           47            26
47           48           113
48           49            47
49           50           453
50           51            29
51           52           169
52           53            35
53           54           619
54           55            87
55           56           145
56           57            72
57           58            97
58           59            92
59           60            67
60           61            87
61           62            80
62           63           504
63           64           135
64           65           132
65           66            23
66           67            81
67           68            70
68           69            50
69           70           104
70           71           437
71           72            47
72           73            24
73           74            48
74           75           311
75           76            32
76           77           203
77           78            41
78           79            93
79           80           128
80           81            27
81           82           131
82           83            39
83           84            64
84           85            30
85           86           267
86           87           176
87           88            11
88           89           239
89           90           670
90           91           211
91           92            20
92           93            31
93           94            56
94           95           144
95           96           143
96           97            54
97           98            50
98           99           211
99          100            27
100         101           170
101         102            59
102         103            77
103         104            18
104         105            14
105         106            41
106         107           130
107         108            69
108         109            69
109         110           347
110         111            56
111         112           158
112         113             5
113         114            66
114         115            79
115         116            25
116         117           120
117         118           165
118         119            22
119         120             7
120         121            55
121         122           120
122         123            63
123         124            79
124         125           182
125         126           105
126         127           305
127         128            91
128         129            64
129         130            37
130         131            59
131         132           116
132         133   

In [13]:
def transform(df):
    transform_data = df[['user_id','order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order',
                                'add_to_cart_order', 'reordered', 'department', 'aisle']]
    #transform_data.set_index = ['user_id']
    product_number = pd.DataFrame(df.groupby(['user_id'])['product_name'].count()).reset_index()
    transform_data2 = transform_data.groupby('user_id').agg({'order_number' : lambda x: x.max(),
                                                             'order_dow': lambda x: x.mode()[0],
                                                             'order_hour_of_day': lambda x: x.mode()[0],
                                                        'days_since_prior_order' : lambda x: round(x.mode()[0]),
                                                        'department' : lambda x: x.mode()[0],
                                                            'aisle' : lambda x: x.mode()[0]}).reset_index()
    transform_data3 = transform_data2.merge(product_number,on='user_id')
    #              merge(cluster_time, on = 'user_id')
    transform_data3 = transform_data3.rename(columns = {'product_name':'product_number'})
    transform_data3.index = transform_data3.user_id
    #transform_data3 = transform_data3.drop(columns=['user_id'])
    #transform_data4 = pd.get_dummies(transform_data3, prefix=['day', 'time', 'department'], columns=['day', 'time', 'department'])
    return transform_data3

In [14]:
transform_prior = transform(prior_data_da)
transform_train = transform(train_data_da)

In [51]:
transform_prior.head()

user_id                       aisle  days_since_prior_order  \
user_id                                                                
1              1                 soft drinks                    30.0   
2              2                      yogurt                     8.0   
3              3                fresh fruits                     7.0   
4              4                frozen meals                    21.0   
5              5  packaged vegetables fruits                    19.0   

         department  order_hour_of_day  order_dow  order_number  \
user_id                                                           
1            snacks          10.542373   2.644068            10   
2        dairy eggs          10.441026   2.005128            14   
3           produce          16.352273   1.011364            12   
4         beverages          13.111111   4.722222             5   
5           produce          15.729730   1.621622             4   

         product_number  
user_id                  
1                    59  
2                   195  
3                    88  
4                    18  
5                    37

### Poisson Model

In [58]:
f = 'days_since_prior_order ~ order_number + product_number + order_dow + order_hour_of_day  + C(department) + C(aisle)'

glm_poisson = smf.poisson(str(f), data = transform_prior)
poisson_res = glm_poisson.fit()

Optimization terminated successfully.
         Current function value: 5.183451
         Iterations 29


In [29]:
transform_train.head(100)

user_id                          aisle  days_since_prior_order  \
user_id                                                                   
1              1                    soft drinks                    14.0   
2              2                   fresh fruits                    30.0   
5              5                    asian foods                     6.0   
7              7                   refrigerated                     6.0   
8              8               fresh vegetables                    10.0   
9              9                         yogurt                    30.0   
10            10                     buns rolls                    30.0   
13            13                          cream                     8.0   
14            14     marinades meat preparation                    11.0   
17            17                    paper goods                    30.0   
18            18                   fresh fruits                     7.0   
21            21           energy sports drinks                    28.0   
23            23            soup broth bouillon                    30.0   
24            24                  juice nectars                     0.0   
27            27                         yogurt                     4.0   
29            29             baking ingredients                    13.0   
30            30  water seltzer sparkling water                    22.0   
34            34                          cream                    30.0   
37            37               fresh vegetables                    16.0   
38            38               fresh vegetables                    30.0   
41            41               fresh vegetables                    30.0   
42            42                candy chocolate                    14.0   
43            43                   fresh fruits                    26.0   
44            44         hot dogs bacon sausage                    30.0   
46            46               fresh vegetables                    11.0   
47            47                         beauty                    16.0   
48            48               fresh vegetables                    14.0   
49            49                  ice cream ice                     2.0   
50            50                   fresh fruits                     7.0   
52            52                   fresh fruits                     3.0   
53            53  water seltzer sparkling water                    13.0   
55            55                   fresh fruits                    30.0   
56            56                   fresh fruits                     6.0   
59            59              baby food formula                    10.0   
62            62                   fresh fruits                    29.0   
63            63                   fresh fruits                    13.0   
64            64                   fresh fruits                    27.0   
65            65               fresh vegetables                    30.0   
66            66             baking ingredients                    23.0   
67            67                   fresh fruits                     5.0   
70            70                   fresh fruits                    30.0   
71            71                        spreads                    17.0   
72            72                           milk                    30.0   
74            74                      dry pasta                    10.0   
76            76                dish detergents                    14.0   
78            78                   fresh fruits                     7.0   
79            79                   fresh fruits                    30.0   
80            80                   fresh fruits                     8.0   
81            81                   fresh fruits                     8.0   
82            82                          cream                    13.0   
83            83                          bread                    30.0   
84            84                 c

In [59]:
y_pred_pois = poisson_res.predict(transform_train.drop(columns = ['days_since_prior_order', 'user_id']))

In [60]:
sm.tools.eval_measures.rmse(transform_train4.days_since_prior_order, y_pred_pois)

9.936890022121226

In [33]:
transform_copy = transform_train.copy()

In [62]:
transform_copy['pred_poisson'] = y_pred_pois

In [30]:
def subset_rmse(n1, n2, prediction, s):
    a = transform_copy[(transform_copy.order_number > n1) & (transform_copy.order_number <= n2)]
    transform_copy.loc[(transform_copy.order_number > n1) & (transform_copy.order_number <= n2), 'category'] =s 
    return sm.tools.eval_measures.rmse(a.days_since_prior_order, a[prediction])

In [103]:
poisson_rmse1 = subset_rmse(0, 20, 'pred_poisson', 'order number <= 20')
poisson_rmse1

10.379606930110672

In [104]:
poisson_rmse2 = subset_rmse(20, 40, 'pred_poisson', '20 < order number <= 40 ')
poisson_rmse2

9.332780364488583

In [105]:
poisson_rmse3 = subset_rmse(40, 60, 'pred_poisson', '40 < order number <= 60 ')
poisson_rmse3

7.098007606522937

In [107]:
poisson_rmse4 = subset_rmse(60, 80, 'pred_poisson', '60 < order number <= 80')
poisson_rmse4

5.958298641268912

In [108]:
poisson_rmse5 = subset_rmse(80, 100, 'pred_poisson', '80 < order number <= 100')
poisson_rmse5

4.321564459588711

In [11]:
t4_g = graphlab.SFrame(transform_copy)
t4_g['category'].show()

NameError: name 'transform_copy' is not defined

### Poisson with GAM

In [70]:
transform_prior_2 = pd.get_dummies(transform_prior4, prefix=['department', 'aisle'], columns=['department', 'aisle'])
transform_train_2 = pd.get_dummies(transform_train4, prefix=['department', 'aisle'], columns=['department', 'aisle'])

In [78]:
X = transform_prior_2.drop(columns = ['days_since_prior_order', 'user_id'])
X2 = transform_train_2.drop(columns = ['days_since_prior_order', 'user_id', 'category'])

In [73]:
gam2 = PoissonGAM().fit(X, transform_prior_2.days_since_prior_order)

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 0

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 1

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 6

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 7

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 8

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 9

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 10

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187:

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 71

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 72

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 73

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 74

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 75

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 76

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 77

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 139

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 140

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 141

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 142

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 143

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 144

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/pygam.py:1187: UserWarning:

detected catergorical data for feature 145

/Users/mengruiyin/anaconda/lib/python2.7/site-packages/pygam/p

In [79]:
predict_gam2 = gam2.predict(X2)
sm.tools.eval_measures.rmse(transform_train_2.days_since_prior_order, predict_gam2)

10.010355148734828

In [80]:
XX = generate_X_grid(gam2)
plt.rcParams['figure.figsize'] = (28, 8)
fig, axs = plt.subplots(1, 4)
titles = X.columns
for i, ax in enumerate(axs):
    pdep, confi = gam2.partial_dependence(XX, feature=i+1, width=.95)
    ax.plot(XX[:, i], pdep)
    ax.plot(XX[:, i], confi[0][:, 0], c='grey', ls='--')
    ax.plot(XX[:, i], confi[0][:, 1], c='grey', ls='--')
    ax.set_title(titles[i])
plt.show()

### Gradient Boosting regression

In [15]:
X_prior = build_feature(transform_prior)
X_train = build_feature(transform_train)
y_prior = transform_prior.days_since_prior_order
y_train = transform_train.days_since_prior_order

In [28]:
gbrt = GradientBoostingRegressor(n_estimators=10, learning_rate = 0.1, max_depth = 5)
gbrt.fit(X_prior, y_prior)
y_pred = gbrt.predict(X_train)

In [22]:
sm.tools.eval_measures.rmse(y_train, y_pred)

9.869582623135328

In [18]:
param_grid = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
              'max_depth': [3,4,5,6,7]}
est = GradientBoostingRegressor(n_estimators = 10)
n_jobs = 4
gs_cv = GridSearchCV(est, param_grid, n_jobs = n_jobs ).fit(X_prior, y_prior)
gs_cv.best_params_

{'learning_rate': 0.1, 'max_depth': 7}

In [34]:
transform_copy['pred_gb'] = y_pred

In [102]:
gb_rmse1 = subset_rmse(0, 20, 'pred_gb', 'order number < 20')
gb_rmse1

10.380034662373173

In [110]:
gb_rmse1 = subset_rmse(20, 40, 'pred_gb', '20 < order number <= 40')
gb_rmse1

8.980089961036517

In [114]:
gb_rmse2 = subset_rmse(40, 60, 'pred_gb', '40 < order number <= 60 ')
gb_rmse2

6.578641123269928

In [115]:
gb_rmse3 = subset_rmse(60, 80, 'pred_gb', '60 < order number <= 80 ')
gb_rmse3

5.790136000912753

In [35]:
gb_rmse4 = subset_rmse(80, 100, 'pred_gb', '80 < order number < 100 ')
gb_rmse4

5.229066474330517

The rmse for gradient boosting is 9.869582623135328, smaller than poisson and poisson with GAM. However, 9.87 is not a good result. So I checked rmse for customers with different number of orders and assumed as number of order increase, the rmse should be smaller. The results followed my assumptions. The rmse for order number less than 20 is 10.38 very large and the rmse for order number greater than 80 is 5.22, much better. However, since in the data, 75% customers have order number less than 20, so the rmse for the whole data is 9.87. 